# Setting

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

<ipython-input-3-452873ea8d09>:7: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [ ]:
# 랜덤 시드 고정
import random
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)

In [ ]:
# DataFrame의 모든 행을 출력하도록 설정
pd.set_option('display.max_rows', None)

In [ ]:
# 출력 설정을 기본값으로 재설정
pd.reset_option('display.max_rows')

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KUBIG/23-2 KDIS 장기 프로젝트 분반/train_1114.csv')
train.head()

,Country Code,Year,Net ODA,"Agriculture, forestry, and fishing, value added (% of GDP)",Average wages,Business extent of disclosure index (0-10),CO2 emissions (metric tons per capita),"Central government debt, total (% of GDP)",Current account balance,Ease of doing business rank,...,Renewable energy consumption,Strength of legal rights index (0-12),Survival to 65_female,Survival to 65_male,Tax revenue (% of GDP),Trade,Unemployment rate,Urban population,women parliaments,Y
0,AUS,2000,2316.780940,3.112843,48213.088676,NaN,17.837318,29.545729,-4.101457,NaN,...,8.42,NaN,91.023514,84.879325,23.042014,40.966987,6.285546,84.235,22.972973,2219.462298
1,AUS,2001,2219.462298,3.499666,48693.302296,NaN,17.930727,27.713129,-2.232985,NaN,...,8.37,NaN,91.378199,85.266528,24.875654,44.250356,6.742173,84.100,NaN,2321.858738
2,AUS,2002,2321.858738,3.946780,49140.631292,NaN,18.123211,25.897645,-3.770657,NaN,...,8.74,NaN,91.232211,85.746027,23.560977,41.471680,6.368911,84.222,25.333333,2328.362733
3,AUS,2003,2328.362733,2.888274,49849.375153,NaN,17.894539,25.086646,-5.312615,NaN,...,7.15,NaN,91.619730,86.086290,24.312503,40.222301,5.928420,84.343,25.333333,2373.256239
4,AUS,2004,2373.256239,3.062690,51414.318393,NaN,18.368871,22.873486,-6.261556,NaN,...,6.68,NaN,91.892235,86.452861,24.271439,37.029712,5.396734,84.463,24.666667,2521.249521


# 결측치 처리

In [ ]:
# 각 컬럼의 결측치 비율 계산
missing_ratio = []
for col in train.columns:
    percent_missing = 100 * (train[col].isnull().sum() / train.shape[0])
    missing_ratio.append((col, percent_missing))

# 결측치 비율이 낮은 순으로 정렬
missing_ratio_sorted = sorted(missing_ratio, key=lambda x: x[1])

for col, percent_missing in missing_ratio_sorted:
    # 결측치 비율이 0보다 큰 경우에만 출력
    if percent_missing > 0:
        msg = 'column: {:>70}\t Percent of NaN value: {:.2f}%'.format(col, percent_missing)
        print(msg)

column:                                                      ICT goods exports	 Percent of NaN value: 0.16%
column:                                                  Life expectancy at 65	 Percent of NaN value: 0.16%
column:                                               Life expectancy at birth	 Percent of NaN value: 0.16%
column:                                                            FDI inflows	 Percent of NaN value: 0.33%
column:                                                           FDI outflows	 Percent of NaN value: 0.33%
column:             Agriculture, forestry, and fishing, value added (% of GDP)	 Percent of NaN value: 0.49%
column:              Industry (including construction), value added (% of GDP)	 Percent of NaN value: 0.49%
column:                                                      women parliaments	 Percent of NaN value: 0.82%
column:                                                         GNI per capita	 Percent of NaN value: 0.98%
column:                     

In [ ]:
#결측치 비율이 50% 이상인 것은 삭제
half_count = len(train) / 2
train = train.dropna(thresh=half_count, axis=1)

print(len(train.columns))

55


- 행 수가 적은 편(600개 정도)을 고려하여, 결측치 비율이 50%인 열들에 대해서는 행 삭제가 아니라 대체(imputation) 기법을 활용할 예정.
- 연도별(2000~2021) 데이터이므로 단순 mean값이나 medain 값으로 대체하는 것이 아니라, 앞뒤 행(전년도, 다음연도)의 영향을 받을 것이라 생각
- 따라서 선형회귀 모형의 추정값으로 결측치를 대체하는 방법을 찾아봄

선형회귀 모형 추정값으로 결측치 대체

*   https://rfriend.tistory.com/636
*   진행 방법
      1.   특정 국가의 데이터 선택
      2.   선형 회귀 모델을 생성하고, 결측치가 없는 연도 데이터로 모델 학습
      3.   결측치가 있는 연도에 대해 모델을 사용하여 값을 예측함
      4.   예측된 값을 원래 데이터프레임의 결측치에 대체
      5.   모든 국가에 대해 반복

In [ ]:
# 원본 데이터 보존
df_train = train.copy()

In [ ]:
from sklearn.linear_model import LinearRegression

def fill_missing_values_with_regression(df_train, country_code, year_col, column_name):
    """
    df_train: 데이터프레임
    country_code: 국가 이름
    year_col: 연도 이름
    column_name: 결측치를 채울 열 이름
    """
    # 해당 국가의 데이터 선택
    country_data = df_train[df_train[country_code] == country]

    # 해당 국가의 모든 연도에 값이 결측된 경우에는 함수를 종료
    if country_data[column_name].notnull().sum() < 1:
        return df_train

    # 연도와 해당 값을 각각 X, y에 할당(NaN이 아닌 값만)
    X = country_data[year_col][country_data[column_name].notnull()].values.reshape(-1, 1)
    y = country_data[column_name][country_data[column_name].notnull()]

    # 선형 회귀 모델 생성 및 학습
    model = LinearRegression()
    model.fit(X, y)

    # 결측치가 있는 연도를 찾아 예측
    X_missing = country_data[year_col][country_data[column_name].isnull()].values.reshape(-1, 1)
    if X_missing.size > 0:
        y_pred = model.predict(X_missing)
        # 예측된 값을 원래 데이터프레임의 결측치에 대체
        df_train.loc[(df_train[country_code] == country) & (df_train[column_name].isnull()), column_name] = y_pred

    return df_train

# 모든 국가와 모든 열에 대해 반복
for country in df_train['Country Code'].unique():
    for column in df_train.columns.drop(['Country Code', 'Year', 'Net ODA']):
        # 결측치가 있는 경우에만 위 함수 호출
        if df_train[column].isnull().any():
            df_train = fill_missing_values_with_regression(df_train, 'Country Code', 'Year', column)

In [ ]:
# 각 컬럼의 결측치 비율 계산
missing_ratio = []
for col in df_train.columns:
    percent_missing = 100 * (df_train[col].isnull().sum() / df_train.shape[0])
    missing_ratio.append((col, percent_missing))

# 결측치 비율이 낮은 순으로 정렬
missing_ratio_sorted = sorted(missing_ratio, key=lambda x: x[1])

for col, percent_missing in missing_ratio_sorted:
    msg = 'column: {:>70}\t Percent of NaN value: {:.2f}%'.format(col, percent_missing)
    print(msg)

column:                                                           Country Code	 Percent of NaN value: 0.00%
column:                                                                   Year	 Percent of NaN value: 0.00%
column:                                                                Net ODA	 Percent of NaN value: 0.00%
column:             Agriculture, forestry, and fishing, value added (% of GDP)	 Percent of NaN value: 0.00%
column:                                                          Average wages	 Percent of NaN value: 0.00%
column:                             Business extent of disclosure index (0-10)	 Percent of NaN value: 0.00%
column:                                 CO2 emissions (metric tons per capita)	 Percent of NaN value: 0.00%
column:                                                Current account balance	 Percent of NaN value: 0.00%
column:                                                     Education spending	 Percent of NaN value: 0.00%
column:                     

In [ ]:
# DataFrame의 모든 행을 출력하도록 설정
pd.set_option('display.max_rows', None)

In [ ]:
# Education spending	 Percent of NaN value: 38.99%
# 제대로 결측치 대체했는지 값 비교하기
comparing_df = pd.DataFrame()

comparing_df['Country Code'] = train['Country Code']
comparing_df['Year'] = train['Year']

comparing_df['Education spending_before'] = train['Education spending']
comparing_df['Education spending_after'] = df_train['Education spending']
print(comparing_df)

    Country Code  Year  Education spending_before  Education spending_after
0            AUS  2000                   9.013473                  9.013473
1            AUS  2001                        NaN                  9.037305
2            AUS  2002                        NaN                  9.131015
3            AUS  2003                        NaN                  9.224725
4            AUS  2004                        NaN                  9.318436
5            AUS  2005                   9.228042                  9.228042
6            AUS  2006                        NaN                  9.505857
7            AUS  2007                        NaN                  9.599567
8            AUS  2008                   9.217424                  9.217424
9            AUS  2009                  10.078766                 10.078766
10           AUS  2010                  10.615802                 10.615802
11           AUS  2011                  10.015199                 10.015199
12          

In [ ]:
# 국가별로 비교해보니 나름 imputation이 잘 된 것 같다는 생각...
# 결측치 제거가 제대로 되지 않은 열에 대해서도 확인
# 제대로 결측치 대체했는지 값 비교하기
comparing_df = pd.DataFrame()

comparing_df['Country Code'] = train['Country Code']
comparing_df['Year'] = train['Year']

comparing_df['government debt_before'] = train['Central government debt, total (% of GDP)']
comparing_df['government debt_after'] = df_train['Central government debt, total (% of GDP)']
print(comparing_df)
# ITA, DEU, NLD처럼 모든 연도가 다 결측치라서 값이 존재하는 연도를 기준으로 결측치를 채울 수 없는 국가는 imputation이 생략됨

    Country Code  Year  government debt_before  government debt_after
0            AUS  2000               29.545729              29.545729
1            AUS  2001               27.713129              27.713129
2            AUS  2002               25.897645              25.897645
3            AUS  2003               25.086646              25.086646
4            AUS  2004               22.873486              22.873486
5            AUS  2005               22.418550              22.418550
6            AUS  2006               21.453913              21.453913
7            AUS  2007               20.109365              20.109365
8            AUS  2008               18.187886              18.187886
9            AUS  2009               23.872061              23.872061
10           AUS  2010               28.670102              28.670102
11           AUS  2011               30.098710              30.098710
12           AUS  2012               39.456437              39.456437
13           AUS  20

In [ ]:
# 출력 설정을 기본값으로 재설정
pd.reset_option('display.max_rows')

In [ ]:
# 남은 결측치에 대해서는 전부 column의 평균값으로 대체함
df_train.fillna(df_train.mean(), inplace=True)

In [ ]:
# Y가 있어 Net ODA 칼럼은 필요없으므로 drop
df_train = df_train.drop('Net ODA', axis=1)

# Scaling

*   다양한 정보를 담은 열이 수십개씩 있으므로, 열마다 단위가 달라 스케일링이 필요하다는 가정
*   스케일러별 성능 비교를 통해 Robust 스케일러로 확정

**Robust Scaler**

*   모든 피처가 같은 크기를 갖는다는 점이 standard와 유사
*   그러나 평균과 분산 대신 특성들의 중앙값을 0, IQE를 1로 스케일링 함
*   Standard와 달리 이상치에 영향을 받지 않음


In [ ]:
#Robust Scaler 사용한 데이터프레임 생성
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()

features = df_train.drop(columns=['Country Code', 'Year', 'Y'])
scaled_features = pd.DataFrame(scaler.fit_transform(features), columns=features.columns)

df_robust = pd.concat([df_train[['Country Code', 'Year', 'Y']], scaled_features], axis=1)

df_robust.head()

,Country Code,Year,Y,"Agriculture, forestry, and fishing, value added (% of GDP)",Average wages,Business extent of disclosure index (0-10),CO2 emissions (metric tons per capita),"Central government debt, total (% of GDP)",Current account balance,Education spending,...,Poverty rate,R&D spending,Renewable energy consumption,Survival to 65_female,Survival to 65_male,Tax revenue (% of GDP),Trade,Unemployment rate,Urban population,women parliaments
0,AUS,2000,2219.462298,0.956576,-0.059905,0.333333,2.441372,-1.621915,-0.648804,0.059523,...,0.933360,-0.292252,-0.249287,-0.241399,-0.045461,0.121605,-0.642888,0.004925,0.346513,-0.262986
1,AUS,2001,2321.858738,1.239016,-0.036629,0.333333,2.464597,-1.682746,-0.391344,0.068333,...,0.904456,-0.077703,-0.252139,-0.105934,0.024267,0.315155,-0.586184,0.124045,0.335552,-0.227508
2,AUS,2002,2328.362733,1.565478,-0.014947,0.333333,2.512456,-1.743008,-0.603222,0.102975,...,0.875552,-0.173910,-0.231033,-0.161691,0.110615,0.176384,-0.634172,0.026672,0.345457,-0.130258
3,AUS,2003,2373.256239,0.792606,0.019406,0.333333,2.455599,-1.769928,-0.815691,0.137617,...,0.846648,-0.058621,-0.321734,-0.013685,0.171890,0.255711,-0.655749,-0.088238,0.355281,-0.130258
4,AUS,2004,2521.249521,0.919957,0.095259,0.333333,2.573537,-1.843391,-0.946447,0.172259,...,0.817744,-0.118571,-0.348545,0.090394,0.237902,0.251377,-0.710886,-0.226939,0.365024,-0.167746


In [ ]:
#중앙값이 모두 0으로 스케일링 된 것을 확인
df_robust.describe()

,Year,Y,"Agriculture, forestry, and fishing, value added (% of GDP)",Average wages,Business extent of disclosure index (0-10),CO2 emissions (metric tons per capita),"Central government debt, total (% of GDP)",Current account balance,Education spending,Employment rate,...,Poverty rate,R&D spending,Renewable energy consumption,Survival to 65_female,Survival to 65_male,Tax revenue (% of GDP),Trade,Unemployment rate,Urban population,women parliaments
count,613.000000,613.000000,613.000000,613.000000,613.000000,613.000000,6.130000e+02,613.000000,613.000000,613.000000,...,613.000000,613.000000,613.000000,613.000000,613.000000,613.000000,613.000000,613.000000,613.000000,613.000000
mean,2010.546493,4975.858994,0.184714,-0.047932,-0.298346,0.253514,-1.390948e-16,-0.009994,-0.008266,-0.088110,...,0.106608,0.067715,0.337761,-0.145966,-0.242302,-0.126892,0.301028,0.275797,-0.173736,-0.030765
std,6.329711,7793.117667,0.990237,0.682586,0.875362,1.019942,1.225992e+00,0.772635,0.765879,0.721155,...,0.676334,0.602672,0.989051,0.833310,0.960319,0.638379,1.031095,1.073313,0.886760,0.600262
min,2000.000000,16.409848,-1.173882,-1.602806,-3.533333,-1.222051,-2.131965e+00,-3.186528,-3.571844,-2.114850,...,-1.124749,-1.113647,-0.690245,-3.242243,-3.904411,-1.476333,-1.012600,-1.139130,-2.130470,-1.225241
25%,2005.000000,466.612603,-0.525845,-0.474475,-0.666667,-0.470412,-7.550928e-01,-0.509409,-0.550934,-0.578141,...,-0.417102,-0.437512,-0.321734,-0.533931,-0.595035,-0.672789,-0.333613,-0.421739,-0.550621,-0.505139
50%,2011.000000,2492.766719,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2016.000000,5325.434068,0.474155,0.525525,0.333333,0.529588,2.449072e-01,0.490591,0.449066,0.421859,...,0.582898,0.562488,0.678266,0.466069,0.404965,0.327211,0.666387,0.578261,0.449379,0.494861
max,2021.000000,51446.363506,4.511355,1.455414,1.252381,4.374086,5.779484e+00,2.186001,4.045267,1.738315,...,2.239079,2.082244,4.183202,1.663810,1.403691,1.659624,5.352541,5.623912,1.473573,1.209942


In [ ]:
df_robust.to_csv('/content/drive/MyDrive/Colab Notebooks/KUBIG/23-2 KDIS 장기 프로젝트 분반/df_robust', index = None)

# EDA
- boxplot, heatmap 등 몇가지 기본적인 EDA 해봤는데 변수가 너무 많아 가독성이 떨어짐 -> 삭제
- 주요 파악 내용 : 칼럼별 이상치가 많고, 상관성이 높은 변수들이 존재함